# Vertex AI Pipelines: pipeline control structures using the KFP SDK

[[github](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/control_flow_kfp.ipynb) - 8b01919]

In [ ]:
import json

import google.cloud.aiplatform as aip
import kfp
from kfp import dsl
from kfp.v2.dsl import component

## Define pipeline

In [ ]:
@component
def args_generator_op() -> str:
    import json

    return json.dumps(
        [{"cats": "1", "dogs": "2"}, {"cats": "10", "dogs": "20"}],
        sort_keys=True
    )

@component
def print_op(msg: str):
    print(msg)

@component
def flip_coin_op() -> str:
    import random

    result = "heads" if random.randint(0, 1) == 0 else "tails"
    return result

@dsl.pipeline(name="control")
def pipeline(
    json_string: str = json.dumps(
        [
            {
                "snakes": "anaconda",
                "lizards": "anole",
                "bunnies": [{"cottontail": "bugs"}, {"cottontail": "thumper"}],
            },
            {
                "snakes": "cobra",
                "lizards": "gecko",
                "bunnies": [{"cottontail": "roger"}],
            },
            {
                "snakes": "boa",
                "lizards": "iguana",
                "bunnies": [
                    {"cottontail": "fluffy"},
                    {"fuzzy_lop": "petunia", "cottontail": "peter"},
                ],
            },
        ],
        sort_keys=True,
    )
):
    flip1 = flip_coin_op()

    with dsl.Condition(flip1.output != "no-such-result", name="alwaystrue"):  # always true
        args_generator = args_generator_op()

        with dsl.ParallelFor(args_generator.output) as item:
            with dsl.Condition(flip1.output == "heads", name="heads"):
                print_op(item.cats)

            with dsl.Condition(flip1.output == "tails", name="tails"):
                print_op(item.dogs)

    with dsl.ParallelFor(json_string) as item:
        with dsl.Condition(item.snakes == "boa", name="snakes"):
            print_op(item.snakes)
            print_op(item.lizards)
            print_op(item.bunnies)

    # with dsl.ParallelFor(json_string) as item:
    #     with dsl.ParallelFor(item.bunnies) as item_bunnies:
    #         print_op(item_bunnies.cottontail)

## Run the pipeline locally

In [ ]:
kfp_endpoint = "http://localhost:8080/pipeline"

kfp.Client(host=kfp_endpoint).create_run_from_pipeline_func(
    pipeline,
    arguments={},
    mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE,
    enable_caching=False)